In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
import os

In [2]:
if 'spark' in locals() and spark!=None:
    spark.stop()


spark=None

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 17:49:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/20 17:49:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
chd = spark.read \
    .option("delimiter", "\t") \
    .csv("/home/mcn26/varef/scripts/noon_data/CHD_MPRA_library_noRC.tsv", header=True)

In [11]:
chd.distinct().count()

14261

In [12]:
chd.count()

14524

In [17]:
chd.columns

['twist_seq', 'seq', 'id', 'allele']

In [18]:
# Group by all columns and count occurrences
grouped_df = chd.groupBy(chd.columns).count()

# Filter rows where count is greater than 1 (i.e., duplicated rows)
duplicated_rows_df = grouped_df.filter(F.col("count") > 1).drop("count")

In [21]:
duplicated_rows_df.show()

+--------------------+-----------+-------+------+
|           twist_seq|        seq|     id|allele|
+--------------------+-----------+-------+------+
|ACTGGCCGCTTGACGGT...|  rs9921666|Non-Ref|     C|
|ACTGGCCGCTTGACGCC...| rs10040387|    Ref|     C|
|ACTGGCCGCTTGACGGT...| rs12102293|    Ref|     T|
|ACTGGCCGCTTGACGCG...| rs70963514|Non-Ref|     T|
|ACTGGCCGCTTGACGGA...| rs12697626|Non-Ref|     A|
|ACTGGCCGCTTGACGCT...| rs77907709|    Ref|     G|
|ACTGGCCGCTTGACGGT...|  rs8043592|Non-Ref|     G|
|ACTGGCCGCTTGACGGG...|   rs729265|    Ref|     G|
|ACTGGCCGCTTGACGTC...| rs12920477|Non-Ref|     G|
|ACTGGCCGCTTGACGCT...|rs111607652|    Ref|     T|
|ACTGGCCGCTTGACGCT...|rs111607652|Non-Ref|TGTTTG|
|ACTGGCCGCTTGACGGA...| rs10050789|    Ref|     C|
|ACTGGCCGCTTGACGCT...|rs111607652|Non-Ref|    TG|
|ACTGGCCGCTTGACGGC...| rs67482039|    Ref|     A|
|ACTGGCCGCTTGACGCG...| rs70963514|Non-Ref|  TAAA|
|ACTGGCCGCTTGACGGG...| rs35424543|Non-Ref|     C|
|ACTGGCCGCTTGACGGG...| rs35424543|Non-Ref|     A|


In [7]:
chd = chd.withColumnRenamed("id", "which")

In [8]:
vcf_schema = T.StructType([
    T.StructField("CHROM", T.StringType(), True),
    T.StructField("POS", T.IntegerType(), True),
    T.StructField("ID", T.StringType(), True),
    T.StructField("REF", T.StringType(), True),
    T.StructField("ALT", T.StringType(), True),
    T.StructField("QUAL", T.StringType(), True),
    T.StructField("FILTER", T.StringType(), True),
    T.StructField("INFO", T.StringType(), True),

])

In [9]:
vcf = spark.read \
    .option("comment", "#") \
    .option("delimiter", "\t") \
    .schema(vcf_schema) \
    .csv("/home/mcn26/varef/data/Malinois/gnomAD_variants/gnomad.genomes.v3.1.2.sites.*.converted.vcf.gz", header=False)

In [22]:
vcf.take(1)

[Row(CHROM='chr2', POS=10185, ID='rs561592428', REF='T', ALT='C', QUAL='.', FILTER='.', INFO='K562__ref=3.00619;HepG2__ref=0.57225424;SKNSH__ref=0.12139538;K562__alt=3.1070452;HepG2__alt=0.5983842;SKNSH__alt=0.17209518;K562__skew=0.100854985;HepG2__skew=0.026129985;SKNSH__skew=0.05069978')]

In [24]:
#drop variants which don't have an rsID
vcf=vcf.filter(vcf.ID != ".")

In [31]:
joined_df = chd.join(vcf, chd.seq == vcf.ID, how="left")

In [ ]:
joined_df.write.option("codec", "org.apache.hadoop.io.compress.GzipCodec") \
    .option("delimiter", "\t") \
    .csv("chd_out.csv.gz", header=True, mode="overwrite")